In [1]:
""" To use Google Drive with Colab, 
1. set use_google_drive to True, and
2. specify a directory in Google Drive (Modify as in your Google Drive)
(You will need to authorize manually.)
"""
use_google_drive = True
workdir = '/content/drive/My Drive/Colab/UCF101_extractor'


import os

try:
    if use_google_drive:
        from google.colab import drive
        drive.mount('/content/drive')
        # Create target directory & all intermediate directories if don't exists
        if not os.path.exists(workdir):
            os.makedirs(workdir)
            print('## Directory: ' , workdir ,  ' was created.') 
        os.chdir(workdir)
        print('## Current working directory: ', os.getcwd())
except:
    print('Run the code without using Google Drive.')
        
try:    
    print('## Check the uptime. (Google Colab reboots every 12 hours)')
    !cat /proc/uptime | awk '{print "Uptime is " $1 /60 /60 " hours (" $1 " sec)"}'
    print('## Check the GPU info')
    !nvidia-smi
    print('## Check the OS') 
    !cat /etc/issue
    print('## Check the Python version') 
    !python --version
    print('## Check the memory')
    !free -h
    print('## Check the disk')
    !df -h
except:
    print('Run the code assuming the environment is not Google Colab.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
## Current working directory:  /content/drive/My Drive/Colab/UCF101_extractor
## Check the uptime. (Google Colab reboots every 12 hours)
Uptime is 0.101733 hours (366.24 sec)
## Check the GPU info
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

## Check the OS
Ubuntu 18.04.2 LTS \n \l

## Check the Python version
Python 3.6.7
## Check the memory
              total        used        free      shared  buff/cache   available
Mem:            12G        448M         10G        868K        1.9G         12G
Swap:            0B          0B          0B
## Check the disk
Filesystem      Size  Used Avail Use% Mounted on
overlay          49G   21G   26G  45% /
tmpfs           6.4G     0  6.4G   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
tmpfs           6.4

In [0]:
import numpy as np
from pathlib import Path 
import cv2
import traceback


top_dir = r'./UCF-101'
f_interval = 5 #25
resize256 = True



top_dir_path = Path(top_dir)

if resize256:
    w_top_dir = top_dir + '_frames' + '256'
else:
    w_top_dir = top_dir + '_frames' 
    
w_top_dir_path = Path(w_top_dir)
print('Top directory to write:', w_top_dir_path)

label_dir_paths = list(top_dir_path.glob('*'))
#label_dir = label_dirs[0]
for label_index, label_dir_path in enumerate(label_dir_paths):
    print('Label: {} | Directory: {}'.format(label_index, label_dir_path))
    w_label_dir_path = w_top_dir_path / label_dir_path.stem
    
    for video_file_path in list(label_dir_path.glob('*')):
        #print(video_file_path)
        try:
            cap = cv2.VideoCapture(video_file_path.as_posix())
            #width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            #height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            #fps = cap.get(cv2.CAP_PROP_FPS)
            #print("width:{}, height:{}, count:{}, fps:{}".format(width,height,count,fps))

            w_video_dir_path = w_label_dir_path / video_file_path.stem

            #print('Video Directory Path to Write:', w_video_dir_path)
            num_triplets = (count // f_interval) // 3
            #print('Number of Triplets:', num_triplets)
            pos_last = num_triplets * 3 * f_interval

            for i in np.arange(0, pos_last, f_interval):
                cap.set(cv2.CAP_PROP_POS_FRAMES, i)
                frame_valid, frame = cap.read()
                if not frame_valid:
                    print('Invalid frame from: {}'.format(video_file_path))
                    break
                if resize256:
                    frame = cv2.resize(frame, (256, 256))

                #triplet_id = 3 * f_interval * ((i // f_interval) // 3)
                #triplet_index = (i // f_interval) % 3            
                #w_image_file_path = w_video_dir_path / 'f_{:04d}/frame_{}.png'.format(triplet_id, triplet_index+1)
                w_image_file_path = w_video_dir_path / 'frame_{:04d}.png'.format(i)         

                w_image_file_path.parent.mkdir(parents=True, exist_ok=True)
                #print('file: ', w_image_file_path)
                cv2.imwrite(w_image_file_path.as_posix(), frame)
        except KeyboardInterrupt:
            break
        except:
            print('### An exception occurred during processing:',video_file_path)
            print(traceback.format_exc())
            print('### Continue running.') 
print('### End of code.')

Top directory to write: UCF-101_frames256
Label: 0 | Directory: UCF-101/Archery
